## 3D Prediction using StarDist

Repo: https://github.com/stardist/stardist?tab=readme-ov-file

## Libraries

In [1]:
import re
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
matplotlib.rcParams['image.interpolation'] = 'none'
%matplotlib inline

from glob import glob
import nibabel as nib
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist import random_label_cmap
from stardist.plot import render_label
from stardist.models import StarDist3D

# Custom packages
from auxiliary import values as v
from auxiliary.gpu import (
    increase_gpu_memory, 
    set_gpu_allocator, 
    clear_session,
    set_mixed_precision
)
from auxiliary import imaging 


### Data

In [2]:
img_name = '20190806_E6_DAPI_decon_0'
img_path = v.data_path + f'/Gr4/RawImages/Nuclei/{img_name}.5.nii.gz'
test_img = imaging.read_nii(img_path)

In [3]:
import cv2 

test_img = test_img.astype(np.uint8).reshape(
    test_img.shape[0], 
    test_img.shape[1], 
    test_img.shape[2]
)

# aux = np.zeros((512, 512, test_img.shape[2]))
# for i in range(test_img.shape[2]):
#     aux[..., i] = cv2.resize(test_img[..., i], (512, 512))
test_img_norm = normalize(test_img, 1, 99.8, axis=(0, 1, 2))

In [4]:
print(test_img_norm.shape)

(1024, 1024, 548)


### Model

In [5]:
# set_mixed_precision()
increase_gpu_memory()
set_gpu_allocator()

In [6]:
from stardist.models import StarDist3D

model_idx = 0
model_names = [
    'n1_stardist_96_(1.6, 1, 1)_(48, 64, 64)_(1, 1, 1)',
    'n2_stardist_96_(1.6, 1, 1)_(48, 64, 64)_(1, 1, 1)',
    'n3_stardist_96_(1.6, 1, 1)_(48, 64, 64)_(1, 1, 1)'
]

model_path = 'models/' + model_names[model_idx] + '/'
model = StarDist3D(None, name=model_names[model_idx], basedir='models')

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.587371, nms_thresh=0.3.


In [8]:
### Predictions and measure metrics
import tensorflow as tf


clear_session()

tf.debugging.set_log_device_placement(True)
strategy = tf.distribute.MirroredStrategy() 

with strategy.scope():
    labels, details = model.predict_instances(test_img_norm)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-07-17 17:17:33.184681: E tensorflow/compiler/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:306] gpu_async_0 cuMemAllocAsync failed to allocate 73551314944 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 3427270656/8169717760
2024-07-17 17:17:33.184723: E tensorflow/compiler/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:311] Stats: Limit:                      8589934592
InUse:                      4601438218
MaxInUse:                   4601438227
NumAllocs:                         150
MaxAllocSize:               2298478592
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2024-07-17 17:17:33.184738: E tensorflow/compiler/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:63] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2024-07-17 17:17:33.184746: E tensorflow/compiler/xla/stream_exe

ResourceExhaustedError: Graph execution error:

Detected at node 'model/down_level_0_no_0/Conv3D' defined at (most recent call last):
    File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/threading.py", line 973, in _bootstrap
      self._bootstrap_inner()
    File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
      self.run()
    File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/keras/engine/training.py", line 2143, in run_step
      outputs = model.predict_step(data)
    File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in predict_step
      return self(x, training=False)
    File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 65, in error_handler
      return fn(*args, **kwargs)
    File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/keras/engine/trainin

Save img

In [ ]:
# segmented_image = nib.Nifti1Image(labels, X[idx].affine)
# file_name = re.search(r'([^/\\]+)(?=\.\w+\.\w+$)', X_names[0]).group()
# nib.save(segmented_image, f'../data/Gr4/Predictions/Stardist/{file_name}.nii.gz')

Plots

In [ ]:
plt.figure(figsize=(16,10))
plt.subplot(121); plt.imshow(test_img, cmap='gray'); plt.axis('off'); plt.title('Raw image')
plt.subplot(122); plt.imshow(render_label(labels, img=test_img)); plt.axis('off'); plt.title('Predicted labels')
plt.tight_layout()
plt.show()

In [ ]:
# plt.figure(figsize=(13,10))
# z = max(0, img.shape[0] // 2 - 5)
# plt.subplot(121)
# plt.imshow((img if img.ndim==3 else img[...,:3])[z], clim=(0,1), cmap='gray')
# plt.title('Raw image (XY slice)')
# plt.axis('off')
# plt.subplot(122)
# plt.imshow((img if img.ndim==3 else img[...,:3])[z], clim=(0,1), cmap='gray')
# plt.imshow(labels[z], cmap=lbl_cmap, alpha=0.5)
# plt.title('Image and predicted labels (XY slice)')
# plt.axis('off');